In [ ]:
!nvidia-smi

Mon Jul 26 10:22:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget -c "https://raw.githubusercontent.com/chef-transformer/chef-transformer/main/data/test-5000.csv" /content/

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import os
import json
import re
import textwrap

import torch
import torch.nn.functional as F
from datasets import load_dataset, load_metric

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [ ]:
model_name_or_path = "mbien/recipenlg"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(model_name_or_path, device)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name_or_path).to(device)

mbien/recipenlg cuda


In [ ]:
dataset = load_dataset("csv", data_files={"test": "/content/test-5000.csv"}, delimiter="\t")["test"]
dataset

Using custom data configuration default-108e22bec8b84362


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /content/cache/csv/default-108e22bec8b84362/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


Dataset({
    features: ['inputs', 'targets'],
    num_rows: 5000
})

In [ ]:
tokenizer("<RECIPE_START>")

{'input_ids': [50265], 'attention_mask': [1]}

In [ ]:
def post_processing(text):
    r_s, r_e = "<RECIPE_START>", "<RECIPE_END>"
    t_s, t_e = "<TITLE_START>", "<TITLE_END>"
    i_s, i_e = "<INGR_START>", "<INGR_END>"
    d_s, d_e = "<INSTR_START>", "<INSTR_END>"

    if not r_s in text or not r_e in text:
        return None
    
    text = text[text.find(r_s) + len(r_s): text.find(r_e)]

    if not t_s in text or not i_s in text or not d_s in text:
        return None
    
    if not t_e in text or not i_e in text or not d_e in text:
        return None

    title = text[text.find(t_s) + len(t_s): text.find(t_e)]
    ingredients = text[text.find(i_s) + len(i_s): text.find(i_e)].replace("<NEXT_INGR>", " ")
    directions = text[text.find(d_s) + len(d_s): text.find(d_e)].replace("<NEXT_INSTR>", " ")

    if not len(title) > 0 or not len(ingredients) > 0 or not len(directions) > 0:
        return None

    title = [w.strip() for w in title.split() if w.strip()]
    ingredients = [w.strip() for w in ingredients.split() if w.strip()]
    directions = [w.strip() for w in directions.split() if w.strip()]

    title = ' '.join(title).lower()
    ingredients = ' '.join(ingredients).lower()
    directions = ' '.join(directions).lower()

    return f"title: {title} ingredients: {ingredients} directions: {directions}"


def post_processing_list(texts):
    new_texts = []
    
    for text in texts:
        text = post_processing(text)
        if text:
            new_texts.append(text)
    
    return new_texts

def predict(batch):
    inputs = [
        '<RECIPE_START> <INPUT_START> ' + inp.replace(', ', ' <NEXT_INPUT> ') + ' <INPUT_END>'
        for inp in batch["inputs"]
    ]
    
    inputs = tokenizer(
        inputs, 
        max_length=256, 
        padding=True, 
        truncation=True, 
        return_tensors='pt'
    )
    input_ids = inputs.input_ids.to(device)
    # attention_mask = inputs.attention_mask.to(device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids, 
            # attention_mask=attention_mask,
            **genkw
        )

    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=False)
    batch["predicted"] = [post_processing_list(outputs[i:i + genkw["num_return_sequences"]]) for i in range(0, len(outputs), genkw["num_return_sequences"])]
    return batch

In [ ]:
# OR You can pass this step
# !wget -c "https://raw.githubusercontent.com/chef-transformer/chef-transformer/main/data/RecipNLG_predicted.json" /content/predicted.json

In [ ]:
sample = dataset.select(range(1))
result = dataset.map(predict, batched=True, batch_size=8)
result

  0%|          | 0/625 [00:00<?, ?ba/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Dataset({
    features: ['inputs', 'predicted', 'targets'],
    num_rows: 5000
})

In [ ]:
df_list = []
for i in tqdm(range(len(result)), position=0):
    inputs, targets, predicted =  result["inputs"][i], result["targets"][i], result["predicted"][i]
    df_list.append({
        "input": inputs,
        "target": targets,
        "predicted": predicted,
    })

df = pd.DataFrame(df_list)
df.to_json("/content/predicted.json")
df.head()

100%|██████████| 5000/5000 [03:00<00:00, 27.66it/s]


,input,target,predicted
0,"margarine, sugar, eggs, sour cream, vanilla, c...",title: sour cream coffee cake ingredients: 1 c...,[]
1,"peanut oil, brown onions, curry powder, cardam...",title: curried spinach onion rice ingredients:...,[title: coconut risotto ingredients: 3 tablesp...
2,"ground chuck, long shuttle noodles, spaghetti ...",title: cavatini casserole ingredients: 2 lb. g...,[]
3,"dates, boiling water, baking mix, sugar, eggs,...",title: quick date nut bread ingredients: 1 1/2...,[title: date-nut cake ingredients: 1 1/2 cups ...
4,"lemon juice, peaches, graham cracker pie crust",title: peach cream pie ingredients: 1 14 oz. c...,[title: cranberry-cherry date-and-peach pie in...


In [ ]:
df = pd.read_json("/content/predicted.json")
print(df.shape)
df["predicted"] = df["predicted"].apply(lambda t: t if len(t) > 0 else None)
df = df.dropna()
df = df.reset_index(drop=True) 
print(df.shape)
df.head()

(5000, 3)
(3050, 3)


,input,target,predicted
0,"peanut oil, brown onions, curry powder, cardam...",title: curried spinach onion rice ingredients:...,[title: coconut risotto ingredients: 3 tablesp...
1,"dates, boiling water, baking mix, sugar, eggs,...",title: quick date nut bread ingredients: 1 1/2...,[title: date-nut cake ingredients: 1 1/2 cups ...
2,"lemon juice, peaches, graham cracker pie crust",title: peach cream pie ingredients: 1 14 oz. c...,[title: cranberry-cherry date-and-peach pie in...
3,"bananas, vanilla wafers, chocolate syrup, pecans",title: banana split cake ingredients: 2 jars m...,[title: spicy peanut butter balls ingredients:...
4,"active dry yeast, sugar, warm water, salt, non...",title: cheese n egg pizza squares ingredients:...,[title: tomato focaccia ingredients: 2 package...


In [ ]:
!pip freeze | grep nltk

nltk==3.6.2


In [ ]:
%%capture
!pip install -U nltk
!pip install jiwer
!pip install rouge_score

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

import re
import math
from collections import Counter
import string
from tqdm import tqdm

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
        
def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)

def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b
    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)
    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

In [ ]:
cosim_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted_list = row["target"], row["predicted"]

    target = " ".join([word.strip() for word in wordpunct_tokenize(target) if word not in string.punctuation and word not in stopwords and word])

    cosim_score = []
    for j in range(len(predicted_list)):
        predicted = " ".join([word.strip() for word in wordpunct_tokenize(predicted_list[j]) if word not in string.punctuation and word not in stopwords and word])
        cosim_score.append(get_result(target, predicted))

    cosim_scores.append(max(cosim_score))
    # break

100%|██████████| 3050/3050 [00:05<00:00, 552.70it/s]


In [ ]:
cosim_score = np.array(cosim_scores).mean()
print(f"Cosine similarity score: {cosim_score}")

Cosine similarity score: 0.5723224779229222


In [ ]:
wer = load_metric("wer")
wer_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted_list = row["target"], row["predicted"]

    wer_score = []
    for j in range(len(predicted_list)):
        score = wer.compute(references=[target], predictions=[predicted_list[j]])
        wer_score.append(score)

    wer_scores.append(min(wer_score))

    # break

100%|██████████| 3050/3050 [00:18<00:00, 164.11it/s]


In [ ]:
wer_score = np.array(wer_scores).mean()
print(f"WER score: {wer_score}")

WER score: 1.2125644379955967


In [ ]:
rouge = load_metric("rouge")
rouge_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predictions = row["target"], row["predicted"]

    references = [target] * len(predictions)
    score = rouge.compute(references=references, predictions=predictions, use_stemmer=True)
    rouge2 = score["rouge2"].high.fmeasure
    rouge_scores.append(rouge2)

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

100%|██████████| 3050/3050 [11:42<00:00,  4.34it/s]


In [ ]:
rouge_score = np.array(rouge_scores).mean()
print(f"Rouge score: {rouge_score}") 

Rouge score: 0.1354438298847996


In [ ]:
import nltk
import nltk.translate.bleu_score as bleu
from nltk.translate.bleu_score import SmoothingFunction

import nltk.translate.gleu_score as gleu
import nltk.translate.meteor_score as meteor


def compute_bleu(recipe, refer):
    hyp = recipe
    refs = refer
    smoothie = SmoothingFunction().method4
    score_ref_a = bleu.sentence_bleu(refs, hyp, smoothing_function=smoothie)
    return score_ref_a


def compute_gleu(recipe, refer):
    hyp = recipe
    refs = refer
    score_ref_a = gleu.sentence_gleu(refs, hyp)
    return score_ref_a

def compute_meteor(recipe, refer):
    hyp = recipe
    refs = refer
    score_ref_a = meteor.meteor_score(refs, hyp)
    return score_ref_a

In [ ]:
bleu_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted = row["target"], row["predicted"]

    target = wordpunct_tokenize(target)
    predicted = [wordpunct_tokenize(p) for p in predicted]

    score = compute_bleu(target, predicted)
    bleu_scores.append(score)

    # break

100%|██████████| 3050/3050 [00:05<00:00, 519.30it/s]


In [ ]:
bleu_score = np.array(bleu_scores).mean()
print(f"BLEU score: {bleu_score}")

BLEU score: 0.11648238226340879


In [ ]:
gleu_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted = row["target"], row["predicted"]

    target = wordpunct_tokenize(target)
    predicted = [wordpunct_tokenize(p) for p in predicted]

    score = compute_gleu(target, predicted)
    gleu_scores.append(score)

    # break

100%|██████████| 3050/3050 [00:04<00:00, 646.13it/s]


In [ ]:
gleu_score = np.array(gleu_scores).mean()
print(f"GLEU score: {gleu_score}")

GLEU score: 0.15037598819673856


In [ ]:
m_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted = row["target"], row["predicted"]

    score = compute_meteor(target, predicted)
    m_scores.append(score)

    # break

100%|██████████| 3050/3050 [00:58<00:00, 51.81it/s]


In [ ]:
m_score = np.array(m_scores).mean()
print(f"METEOR score: {m_score}")

METEOR score: 0.23099423196957075
